In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA_ext.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA_ext.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined_ext.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined_ext.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA_ext.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA__ext.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA_ext.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("East_Massachusetts_Apr_weekend_ext");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/East_Massachusetts_trips_Apr_weekend_ext.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 484 entries:
  (6,9)   => 33.959264
  (18,3)  => 0.0
  (21,13) => 0.0
  (22,16) => 0.0
  (15,17) => 38.992326
  (22,14) => 0.0
  (13,9)  => 11.759216
  (8,9)   => 0.0
  (9,1)   => 0.0
  (17,14) => 0.0
  (16,6)  => 0.0
  (16,12) => 0.0
  (16,13) => 30.927875
  (22,2)  => 0.0
  (7,16)  => 68.917088
  (1,12)  => 44.532856
  (4,5)   => 0.0
  (11,8)  => 126.276974
  (18,21) => 0.0
  (19,4)  => 0.0
  (20,13) => 0.0
  (15,4)  => 36.68465
  (6,18)  => 0.0
  (5,16)  => 4.264274
  (4,9)   => 0.0
  ⋮       => ⋮

In [7]:
odPairLabel_

Dict{UTF8String,Any} with 462 entries:
  "306" => {15,12}
  "153" => {8,6}
  "1"   => {1,2}
  "54"  => {3,13}
  "101" => {5,18}
  "371" => {18,14}
  "41"  => {2,21}
  "65"  => {4,2}
  "447" => {22,6}
  "335" => {16,21}
  "362" => {18,5}
  "326" => {16,11}
  "299" => {15,5}
  "168" => {8,22}
  "403" => {20,4}
  "159" => {8,13}
  "228" => {11,19}
  "332" => {16,18}
  "190" => {10,1}
  "270" => {13,19}
  "227" => {11,18}
  "223" => {11,14}
  "453" => {22,12}
  "88"  => {5,4}
  "297" => {15,3}
  ⋮     => ⋮

In [8]:
demandsVec = zeros(462)

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

462-element Array{Float64,1}:
 108.218  
 531.044  
   7.69956
   0.0    
 470.499  
  76.3848 
   0.0    
   5.11195
  23.9917 
  39.2044 
  44.5329 
  50.145  
  77.6206 
   ⋮      
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    
   0.0    

In [10]:
for key=keys(odPairRoute)
    if contains(key, "462-")
        println(key)
    end
end

462-329


In [11]:
linkRoute

Dict{UTF8String,Any} with 1729 entries:
  "24-106" => 1.0
  "42-74"  => 1.0
  "42-365" => 1.0
  "60-8"   => 1.0
  "73-260" => 1.0
  "22-355" => 1.0
  "19-180" => 1.0
  "62-421" => 1.0
  "68-199" => 1.0
  "2-47"   => 1.0
  "66-86"  => 1.0
  "50-450" => 1.0
  "7-225"  => 1.0
  "32-306" => 1.0
  "40-13"  => 1.0
  "39-243" => 1.0
  "1-225"  => 1.0
  "41-157" => 1.0
  "28-23"  => 1.0
  "70-83"  => 1.0
  "40-1"   => 1.0
  "50-273" => 1.0
  "8-28"   => 1.0
  "6-291"  => 1.0
  "57-195" => 1.0
  ⋮        => ⋮

In [12]:
deg = 8
c = 0.5
lam = 10000.0

coeffs_dict = readall("../temp_files/coeffs_dict_Apr_weekend_ext.json")
coeffs_dict = JSON.parse(coeffs_dict)
fcoeffs = coeffs_dict["($(deg),$(c),$(lam),1)"]

9-element Array{Any,1}:
  1.0       
 -1.82929e-5
  0.00117008
  0.118513  
 -0.641234  
  1.5373    
 -1.92848   
  1.23275   
 -0.317075  

In [13]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 74
numRoute = 462
numOD = 462

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .15*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
    free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
    free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6 +
    free_flow_time[a] * fcoeffs[8] * linkFlow[a]^8 / capacity[a]^7 +
    free_flow_time[a] * fcoeffs[9] * linkFlow[a]^9 / capacity[a]^8, a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2265
Number of nonzeros in inequality constraint Jacobian.:      462
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:      536
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [14]:
getValue(linkFlow)

74-element Array{Float64,1}:
  645.488      
  399.277      
 1001.54       
 1247.75       
  274.669      
  418.601      
  537.27       
  180.222      
  501.207      
  884.226      
  106.948      
  212.222      
  198.678      
    ⋮          
    4.26427    
  401.022      
  526.453      
  255.7        
    2.73691e-48
   -6.01853e-36
    1.64215e-47
    0.0        
  133.136      
   26.4613     
  198.678      
  312.761      

In [15]:
getObjectiveValue(m)

4005.9467012560654

In [16]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 74 entries:
  (17,15) => 26.461349
  (3,6)   => 501.207144
  (21,13) => 97.454713
  (22,16) => -6.0185310762101094e-36
  (19,12) => 3.2774179127878e-36
  (22,14) => 1.642146637880645e-47
  (15,17) => 133.136196
  (10,11) => 515.5478459999999
  (13,12) => 0.0
  (3,1)   => 1247.752064
  (16,13) => 401.02235800000005
  (4,5)   => 398.79295700000006
  (13,16) => 4.264274
  (17,16) => 312.760696
  (20,13) => 134.300329
  (12,8)  => 433.43222699999995
  (2,4)   => 537.2701949999998
  (16,17) => 198.677762
  (1,2)   => 645.488267
  (14,21) => 97.454713
  (2,3)   => 274.66938200000004
  (13,19) => 1.642146637880645e-47
  (13,14) => 0.0
  (16,14) => 255.699541
  (8,12)  => 422.243388
  ⋮       => ⋮

In [17]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_ext

No dicts found; please check load_dicts...


In [18]:
flow_user = GLS_Apr_weekday_PM_ext.x_

74x30 Array{Float64,2}:
 1366.35    791.95    771.324  …  1015.01        1624.03     280.272
 1206.76    992.338   991.158     1000.59        1273.13     570.559
 2114.24   1499.68   1200.6       1171.78        1608.83     947.273
 2273.84   1299.3     980.768     1186.21        1959.73     656.986
   58.465   201.97    271.021       86.5748        40.0646   261.282
  139.064   247.675   201.826  …    23.5801       146.69     143.578
 1193.7    1571.81   1353.56       781.132       1109.21    1362.48 
  953.514  1726.49   1642.59       829.701        651.679   1770.47 
 2124.9    1398.79    938.436     1533.92        1629.54    1007.64 
 2365.09   1244.1     649.406     1485.35        2087.07     599.646
  406.406   514.047   567.011  …   469.666        455.737    655.187
  281.997  1677.2    1591.06       204.042        239.965   1660.44 
  204.522   246.959  1020.38         1.2451e-13   268.26    1131.08 
    ⋮                          ⋱                                    
  817.703 

In [19]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 + free_flow_time[a] * fcoeffs[8] * linkFlowVec[a]^8 / capacity[a]^7 + free_flow_time[a] * fcoeffs[9] * linkFlowVec[a]^9 / capacity[a]^8 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [20]:
weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

for i in weekend_Apr_list
    println(socialObj(flow_user[:, i])/getObjectiveValue(m))
end

2.228320873204623
1.7229997166590165
2.5672137211663086
1.5630738982529582
2.2404881392729874
1.5379303125577986
2.1757321568107812
1.5618390102283097
1.8089912159467127


In [21]:
length(weekend_Apr_list)

9